## Procesamiento de resultados desde .csv (Scopus)

usamos pandas para importar los datos y extraer la información.

Primero seleccione de la lista el archivo a procesar:

In [5]:
import os
import pandas as pd

i = 0
nombres = []
extensiones = []
for raiz,carpetas,archivos in os.walk("./busquedas"):
    for i,f in enumerate(archivos):
        print(i,"->",archivos[i])
        nombres.append(archivos[i].split(".")[0])
    pass
# pd.read_csv("/busquedas/2_1-100.csv")

0 -> 3_1-100.csv
1 -> 3_1-100_kw.csv
2 -> 4_todas.csv
3 -> 5_todas.csv
4 -> Filtro de documentos por titulo.xlsx


Luego, importelo al cuaderno como dataframe, escoja el índice de acuerdo a lo anterior

In [6]:
indice = 3
resultDf = pd.read_csv(raiz+"/"+archivos[indice])
resultDf

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Cited by,DOI,Link,Author Keywords,Index Keywords,Document Type,Publication Stage,Open Access,Source,EID
0,"Harun N.H., Misron N., Sidek R.M., Aris I., Wa...",55986733800;6505728008;35801306500;6603306751;...,Dual resonant frequencies effects on an induct...,2014,Sensors (Switzerland),14,11,NaN,21923.0,21940.0,...,5.0,10.3390/s141121923,https://www.scopus.com/inward/record.uri?eid=2...,Air coil; Frequency characteristics; Induction...,Fruits; Grading; Inductive sensors; Natural fr...,Article,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-84912018688
1,"Misron N., Harun N.H., Lee Y.K., Sidek R.M., A...",6505728008;55986733800;56023274200;35801306500...,Improvement in sensitivity of an inductive oil...,2014,Sensors (Switzerland),14,2,NaN,2431.0,2448.0,...,9.0,10.3390/s140202431,https://www.scopus.com/inward/record.uri?eid=2...,Air coil; Frequency characteristics; Inductive...,Detection methods; Frequency characteristic; I...,Article,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-84893286586
2,"Aliteh N.A., Misron N., Aris I., Sidek R.M., T...",57192708910;6505728008;6603306751;35801306500;...,Triple flat-type inductive-based oil palm frui...,2018,Sensors (Switzerland),18,8,2496,NaN,NaN,...,5.0,10.3390/s18082496,https://www.scopus.com/inward/record.uri?eid=2...,Air coil; Inductive concept; Maturity classifi...,Capacitance; Fruits; Inductive sensors; Moistu...,Article,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-85052140678
3,"Harun N.H., Misron N., Sidek R.M., Aris I., Ah...",55986733800;6505728008;35801306500;6603306751;...,Investigations on a novel inductive concept fr...,2013,Sensors (Switzerland),13,2,NaN,2254.0,2266.0,...,19.0,10.3390/s130202254,https://www.scopus.com/inward/record.uri?eid=2...,Air coil; Frequency characteristics; Inductive...,Critical factors; Frequency characteristic; Fr...,Article,Final,"All Open Access, Gold, Green",Scopus,2-s2.0-84875179916
4,"Yeow Y.K., Abbas Z., Khalid K.",56023730400;7005822035;7004042433;,Application of microwave moisture sensor for d...,2010,Measurement Science Review,10,1,NaN,7.0,14.0,...,49.0,10.2478/v10048-010-0002-x,https://www.scopus.com/inward/record.uri?eid=2...,Dielectric constant; Microwave sensors; Moistu...,NaN,Review,Final,"All Open Access, Gold",Scopus,2-s2.0-77249095470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,"Modi A.J., Gosai D.C.",55677958400;54792055000;,Experimental study on thermal barrier coated d...,2010,SAE International Journal of Fuels and Lubricants,3,2,NaN,246.0,259.0,...,23.0,10.4271/2010-01-1519,https://www.scopus.com/inward/record.uri?eid=2...,NaN,Adiabatic conditions; Bio-diesel fuel; Brake t...,Article,Final,NaN,Scopus,2-s2.0-79955495567
214,[No author name available],[No author id available],2013 International Conference on Nanoscience a...,2014,Advanced Materials Research,832,NaN,NaN,NaN,NaN,...,NaN,NaN,https://www.scopus.com/inward/record.uri?eid=2...,NaN,NaN,Conference Review,Final,NaN,Scopus,2-s2.0-84891600246
215,"Long Y., Lian Y., Ma M., Song H., He D.",36976012000;57211074543;57205714318;1734295890...,Detection of tomato hardness based on hyperspe...,2019,Nongye Gongcheng Xuebao/Transactions of the Ch...,35,13,NaN,270.0,276.0,...,8.0,10.11975/j.issn.1002-6819.2019.13.032,https://www.scopus.com/inward/record.uri?eid=2...,Algorithms; Characteristic wavelength; Hardnes...,Algorithms; Classification (of information); F...,Article,Final,NaN,Scopus,2-s2.0-85072574492
216,"Wu M., Zhang R., Shi Y., Dong H., Ding T., Zha...",57205416695;57203680739;57219280840;5722099052...,Changes of flesh texture and nutritional compo...,2020,Journal of Fruit Science,37,9,NaN,1404.0,1412.0,...,NaN,10.13925/j.cnki.gsxb.20200106,https://www.scopus.com/inward/record.uri?eid=2...,Apple; Nutritional components; Storage; Textur...,NaN,Article,Final,NaN,Scopus,2-s2.0-85092021122


Ahora puede usar la información, podemos extraer la info de las columnas separadas por caracteres, ignorando espacios vacíos:

In [7]:
def extraerKeywords(df,columna):
    kws = []
    for text in df.loc[:,columna].dropna():
        text = text.split(";")
        for item in text:
            kws.append(item)
    return kws

*Notas*: 
- se debe usar dropna() para quitar los campos que vienen vacíos desde los datos. 
- loc es el método de localizacion por nombre de columna \[: = todos, "---" = nombre de columna\]
- Para separaciones más complejas, usar nltk

```python
from nltk.tokenize import sent_tokenize
print(sent_tokenize(resultDf["Author Keywords"][0]))
```

In [8]:
listaTitulos = resultDf.loc[:,["Title","Year"]]
listaTitulos

,Title,Year
0,Dual resonant frequencies effects on an induct...,2014
1,Improvement in sensitivity of an inductive oil...,2014
2,Triple flat-type inductive-based oil palm frui...,2018
3,Investigations on a novel inductive concept fr...,2013
4,Application of microwave moisture sensor for d...,2010
...,...,...
213,Experimental study on thermal barrier coated d...,2010
214,2013 International Conference on Nanoscience a...,2014
215,Detection of tomato hardness based on hyperspe...,2019
216,Changes of flesh texture and nutritional compo...,2020


In [10]:
# ejemplo para extraer total de keywords

campo1 = "Author Keywords"
campo2 = "Index Keywords"

listaKeywords = extraerKeywords(resultDf,campo1)
listaKeywords.extend(extraerKeywords(resultDf,campo2))
kwDf = pd.DataFrame(listaKeywords)
kwSum = kwDf.value_counts()
# kwSum = kwSum.iloc[0:5]
#kwSum = pd.DataFrame(kwSum,columns = ["Repeticiones"]).reset_index()
#kwSum = kwSum.rename(columns = {0:'Keyword'})
kwSum.to_csv("./busquedas/"+nombres[indice]+"_kw.csv",header = False,sep = ";")
kwSum

 Oil palm                      48
 Palm oil                      48
 Fruits                        40
 Elaeis                        37
 Indonesia                     22
                               ..
 Surface measurement            1
 Surface acoustic wave          1
 Supplementation                1
 Supercapacitor application     1
 (Hi-Bi) waveguide              1
Length: 2668, dtype: int64

Comparar búsqueda de Vanessa y Sebas

In [23]:
indice = 4
# resultDf = pd.read_csv()
busqueda4Df = pd.read_excel(open(raiz+"/"+archivos[indice], 'rb'),
                          sheet_name='Hoja1',
                          index_col=None,
                          skiprows=None,
#                           usecols=range(1,elementosClave+1),
                          keep_default_na=True,
                         )
busqueda4Df["TITULO"]

0     Application of an inductive sensor system for ...
1     Relative estimation of water content for flat-...
2     Improvement in sensitivity of an inductive oil...
3     Triple flat-type inductive-based oil palm frui...
4     Investigations on a novel inductive concept fr...
                            ...                        
86    Time-resolved Reflectance Spectroscopy for the...
87    Handheld, smartphone based spectrometer for ra...
88    Sorting dates fruit bunches based on their mat...
89    Design of hand-operated piggyback jaw gripper ...
90    Detection of tomato hardness based on hyperspe...
Name: TITULO, Length: 91, dtype: object

Remover duplicados

In [14]:
result2Df = resultDf[~resultDf.Title.isin(busqueda4Df.TITULO)]
result2Df.to_csv("./busquedas/"+nombres[indice]+"_diferentes.csv",header = False,sep = ";")

Seleccionar los relevantes

In [22]:
result3Df = result2Df.loc[[19,25,31,33,34,40,43,45,46,52,60,69,90,92,93,98,
               106,108,109,118,119,124,127,129,134,138,145,154,
               160,168,169,175,184,185,186,188,211,215]]
result3Df.to_csv("./busquedas/"+nombres[indice]+"_seleccionadas.csv",header = False,sep = ";")

In [42]:
# result3Df["Source title"].to_csv("./busquedas/"+nombres[indice]+"_seleccionadasDOI.csv",header = False,sep = ";")